In [1]:
import os

# Set env vars BEFORE importing huggingface modules
os.environ["HF_HOME"] = "/projects/sciences/computing/sheju347/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/projects/sciences/computing/sheju347/.cache/huggingface/hub"

# Now import huggingface modules
from huggingface_hub import constants

print("HF_HOME:", constants.HF_HOME)
print("HF_HUB_CACHE:", constants.HF_HUB_CACHE)

HF_HOME: /projects/sciences/computing/sheju347/.cache/huggingface
HF_HUB_CACHE: /projects/sciences/computing/sheju347/.cache/huggingface/hub


In [6]:

class TrainingData:
    def __init__(self, context: str, question: str, is_correct: bool):
        self.context = context
        self.question = question
        self.is_correct = is_correct


In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# model_name = "google/flan-t5-large"  # or t5-base, t5-large
# model_name = "google-t5/t5-base"
# model_name = "google-t5/t5-small"
model_name = "google-t5/t5-large"
# model_name = "/projects/sciences/computing/sheju347/RAG/sentence_level_classifier/t5-large-epoch-10-3/checkpoint-853"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
print("done")

done


In [12]:
import re
import os

# QUESTION_COUNT = 10178

# log_file_list = [
#     "9-20-train_150_30_1st_H100.txt", 
#     "9-20-train_150_30_2nd_H100.txt", 
#     "9-21-train_150_30_3rd_H100.txt",
#     "9-23-train_150_30_4th_H100.txt",
#     "9-24-train_150_30_5th_H100.txt",
#                 ]
log_file_list = ["10-7-150-30-3-sent_level_training_data.txt"]
# log_file_list = ["10-7-sent_level_training_data.txt"]
# log_file = "9-20-train_150_30_2nd_H100.txt"

# path = "../../notebooks/"
path = "./"

file_name_list = [path + log_file for log_file in log_file_list]

training_data_list = []

for file_name in file_name_list:
    with open(file_name) as f:
        is_reading_context = False
        context = ""
    
        is_reading_question = False
        question = ""
    
        for line in f:
            # if line == "\n":
            #     continue
    
            stripped_line = line.strip()
            if stripped_line == "Context:":
                is_reading_context = True
            elif stripped_line == "Question:":
                is_reading_context = False
                is_reading_question = True
            elif stripped_line.startswith("[output]"):
                is_correct = stripped_line[len("[output]"):] == "True"
                context = context.strip()
                question = question.strip()
                data = TrainingData(context, question, is_correct)
                # print("aaa", context, question, is_correct)
                # if len(training_data_list) > 2000:
                #     break
    
                training_data_list.append(data)
    
                # if len(training_data_list) % 100 == 0:
                #     print(f"finished processing {len(training_data_list)} data")
    
                # clear
                is_reading_context = False
                context = ""
                is_reading_question = False
                question = ""
                # break
            else:
                if is_reading_context:
                    context += line
                elif is_reading_question:
                    question += line

# training_data_list = training_data_list[:1000]
print(len(training_data_list))
sample = training_data_list[6]
print(sample.context)
print(sample.question)
print(training_data_list[6].is_correct)

5925
Frozen section investigation revealed the presence of signet cell adenocarcinoma.
A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?

[A] : Ampicillin
[B] : Ceftriaxone
[C] : Doxycycline
[D] : Nitrofurantoin
True


In [14]:
# Force set data to be balanced
adapted_data_list = []
incorrect_count = 0
for data in training_data_list:
    if data.is_correct == False:
        incorrect_count += 1

counter = 0
for data in training_data_list:
    if data.is_correct == False:
        adapted_data_list.append(data)
    else:
        if counter < incorrect_count + 0:
            adapted_data_list.append(data)
            counter += 1

training_data_list = adapted_data_list


# Change the label based on length
lengths = []
for data in training_data_list:
    lengths.append(len(data.context) + len(data.question))
import statistics
median_len = statistics.median(lengths)
print("median_len", median_len)

# for data in training_data_list:
#     if (len(data.context) + len(data.question)) > median_len:
#         data.is_correct = True
#     else:
#         data.is_correct = False
    

false_count = 0
true_count = 0
for data in training_data_list:
    if data.is_correct:
        true_count += 1
    else:
        false_count += 1
print(false_count, true_count)

median_len 908.5
1350 1350


In [45]:
# filename = "./data/cfimdb-train.txt"

# training_data_list = []

# with open(filename, 'r') as fp:
#     for line in fp:
#         label, org_sent = line.split(' ||| ')
#         # sent = org_sent.lower().strip()
#         # tokens = tokenizer.tokenize("[CLS] " + sent + " [SEP]")
#         label = int(label.strip())
#         # if label not in num_labels:
#         #     num_labels[label] = len(num_labels)
#         data = TrainingData(org_sent, "", label)
#         training_data_list.append(data)

# print(training_data_list[6].context, training_data_list[6].is_correct)

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Suppose training_data_list is your list of TrainingData objects
data = pd.DataFrame([{
    "context": d.context,
    "question": d.question,
    "label": int(d.is_correct)   # True->1, False->0
} for d in training_data_list])

# Train/val/test split (e.g., 70/15/15)
train_df, temp_df = train_test_split(data, test_size=0.3, random_state=42, stratify=data["label"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])


# # Don't shuffle
# n = len(data)
# train_end = int(0.7 * n)
# val_end = int(0.85 * n)  # 70% train, 15% val, 15% test

# # Split sequentially (no shuffle)
# train_df = data.iloc[:train_end]
# val_df = data.iloc[train_end:val_end]
# test_df = data.iloc[val_end:]



print(len(train_df), len(val_df), len(test_df))
# print(test_df.iloc[0]["context"])
# print(test_df.iloc[0]["question"])
print(test_df.iloc[1]["label"])

1890 405 405
1


In [7]:
# for i, row in train_df.iterrows():
#     print(f'{row["context"]} {row["label"]}')

In [16]:
import torch
from torch.utils.data import Dataset

MAX_LEN = 1024 #512
TARGET_MAX_LEN = 2

class QADataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=MAX_LEN, target_max_len=TARGET_MAX_LEN):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.target_max_len = target_max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context = self.data.loc[idx, "context"]
        question = self.data.loc[idx, "question"]
        label = self.data.iloc[idx]["label"]

        input_text = f"question: {question} context: {context}"
        target_text = "yes" if label == 1 else "no"

        # Keep return_tensors=None, let __getitem__ return 1D tensors
        encoding = self.tokenizer(
            input_text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors=None
        )
        
        target_encoding = self.tokenizer(
            target_text,
            truncation=True,
            padding="max_length",
            max_length=self.target_max_len,
            return_tensors=None
        )

        # Convert to tensors (1D)
        input_ids = torch.tensor(encoding["input_ids"], dtype=torch.long)
        attention_mask = torch.tensor(encoding["attention_mask"], dtype=torch.long)
        labels = torch.tensor(target_encoding["input_ids"], dtype=torch.long)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

In [17]:
train_dataset = QADataset(train_df, tokenizer)
val_dataset = QADataset(val_df, tokenizer)
test_dataset = QADataset(test_df, tokenizer)
all_dataset = QADataset(data, tokenizer)

In [18]:
print(train_dataset[9]["input_ids"].tolist())
# print(len(train_dataset[2]["input_ids"].tolist()))
# print(train_dataset[1]["labels"])

[822, 10, 71, 3, 3539, 18, 1201, 18, 1490, 2335, 28, 6658, 3, 32, 115, 7593, 757, 3, 26836, 1994, 19, 1940, 12, 8, 3583, 3066, 250, 13, 17055, 6, 643, 3, 14693, 6, 1460, 8284, 6, 11, 3, 9, 2192, 19222, 5, 451, 65, 3, 24198, 80, 4153, 13, 22893, 1444, 21, 604, 203, 68, 10399, 10257, 209, 215, 977, 5, 451, 1342, 28, 160, 3062, 11, 160, 30963, 6, 113, 2467, 7, 239, 2864, 5, 1347, 2912, 19, 220, 20677, 1956, 254, 41, 19621, 1956, 371, 137, 15576, 6498, 1267, 24097, 975, 6959, 75, 5499, 155, 159, 6, 29586, 52, 52, 88, 9, 6, 11, 3, 4203, 532, 3357, 1162, 8760, 1558, 406, 1215, 76, 5522, 7, 5, 9006, 2505, 3606, 7, 7952, 28, 46, 14669, 26, 26429, 2258, 226, 32, 18095, 5, 6863, 13, 3, 9, 2672, 28, 84, 13, 8, 826, 12009, 13, 1041, 19, 167, 2016, 58, 784, 188, 908, 3, 10, 86, 13506, 1575, 13, 206, 2482, 32, 1583, 7211, 20146, 9, 7, 15, 784, 279, 908, 3, 10, 86, 13506, 1575, 13, 23844, 29, 3017, 14836, 784, 254, 908, 3, 10, 86, 13506, 1575, 13, 3, 29, 1238, 9, 7619, 7664, 15, 784, 308, 908, 3, 10,

In [19]:
from transformers import Trainer, TrainingArguments


output_dir = "/projects/sciences/computing/sheju347/RAG/sentence_level_classifier/t5-large-epoch-10-5" # "./results"

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# def compute_metrics(eval_pred):
#     from sklearn.metrics import accuracy_score, f1_score
#     logits, labels = eval_pred
#     preds = logits.argmax(-1)
#     return {
#         "accuracy": accuracy_score(labels, preds),
#         "f1": f1_score(labels, preds)
#     }

def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, f1_score
    import torch

    # Extract logits and labels
    logits = eval_pred.predictions
    labels = eval_pred.label_ids

    # If logits is a tuple (common for seq2seq models), take the first element
    if isinstance(logits, tuple):
        logits = logits[0]

    # Convert logits to predicted token IDs
    pred_ids = torch.argmax(torch.tensor(logits), dim=-1)
    
    # Decode to text
    decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    
    # Map yes/no to 1/0
    y_pred = [1 if x.strip().lower() == "yes" else 0 for x in decoded_preds]
    
    # True labels
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    y_true = [1 if x.strip().lower() == "yes" else 0 for x in decoded_labels]

    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred)
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipykernel_90852/778904191.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.464000,0.434155,0.883951,0.893424
2,0.227200,0.202204,0.911111,0.913462
3,0.224100,0.182523,0.925926,0.929907
4,0.153200,0.171232,0.920988,0.923810
5,0.151000,0.186721,0.930864,0.933333
6,0.146400,0.152296,0.925926,0.928910
7,0.130000,0.162545,0.928395,0.931442
8,0.104000,0.193223,0.930864,0.933014
9,0.124300,0.189722,0.928395,0.929782
10,0.092800,0.201653,0.925926,0.927536


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=9450, training_loss=0.16622769875501198, metrics={'train_runtime': 3186.7065, 'train_samples_per_second': 5.931, 'train_steps_per_second': 2.965, 'total_flos': 8.18387877888e+16, 'train_loss': 0.16622769875501198, 'epoch': 10.0})

In [ ]:
from sklearn.metrics import accuracy_score

# test_ds = all_dataset
# test_ds = Subset(all_dataset, range(800))
# test_ds = Subset(test_dataset, range(800))
# test_ds = val_dataset
test_ds = test_dataset
# test_ds = Subset(train_dataset, range(1000, 3000))
# test_ds = train_dataset
print(len(test_ds))

preds = trainer.predict(test_ds)

print(preds.metrics)  # optional, shows overall metrics

# Get logits
logits = preds.predictions  # <--- use .predictions


# If logits is a tuple (common for seq2seq models), take the first element
if isinstance(logits, tuple):
    logits = logits[0]  # shape: (num_examples, seq_len, vocab_size)

print("logits", logits)
# Convert logits to token IDs
pred_ids = torch.argmax(torch.tensor(logits), dim=-1)

print ("pred_ids", pred_ids)
# Decode token IDs to text
decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(preds.label_ids, skip_special_tokens=True)

print("decoded_preds", decoded_preds)
print("decoded_labels", decoded_labels)

# Convert yes/no to 1/0
y_pred = [1 if x.strip().lower() == "yes" else 0 for x in decoded_preds]
y_true = [1 if x.strip().lower() == "yes" else 0 for x in decoded_labels]

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)


In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import torch
from torch.utils.data import Subset

# trainer.args.predict_with_generate = False  # make sure no .generate()

chunk_size = 500  # adjust depending on your GPU memory
all_logits = []
all_labels = []

for start in range(0, len(test_dataset), chunk_size):
    end = min(start + chunk_size, len(test_dataset))
    subset = Subset(test_dataset, range(start, end))
    preds = trainer.predict(subset)

    logits = preds.predictions
    if isinstance(logits, tuple):
        logits = logits[0]

    all_logits.append(logits)
    all_labels.append(preds.label_ids)

# Concatenate all chunks
logits = np.concatenate(all_logits, axis=0)
labels = np.concatenate(all_labels, axis=0)

print("logits shape:", logits.shape)

# Convert logits to predicted token IDs
pred_ids = torch.argmax(torch.tensor(logits), dim=-1)

# Only take the first token for yes/no classification
first_token_ids = pred_ids[:, 0]

# Decode predictions and labels
decoded_preds = tokenizer.batch_decode(first_token_ids, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

print("decoded_preds sample:", decoded_preds[:20])
print("decoded_labels sample:", decoded_labels[:20])

# Map yes/no to 1/0
y_pred = [1 if x.strip().lower() == "yes" else 0 for x in decoded_preds]
y_true = [1 if x.strip().lower() == "yes" else 0 for x in decoded_labels]

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)


In [ ]:
preds = trainer.predict(test_dataset)
print(preds)
y_pred = preds.predictions.argmax(-1)
y_true = preds.label_ids

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

In [ ]:
y_preds = preds.predictions.argmax(-1)
count_predict_1 = 0
count_label_1 = 0
for i in range(len(test_dataset)):
    if i < 20:
        print(sum(test_dataset[i]["attention_mask"]), test_dataset[i]["labels"], y_preds[i])
    count_predict_1 += y_preds[i]
    count_label_1 += test_dataset[i]["labels"].item()

print(count_predict_1, "/", len(test_dataset), count_predict_1 / len(test_dataset))
print(count_label_1, "/", len(test_dataset), count_label_1 / len(test_dataset))